# AirbnbBarcelona

## Importing libraries

In [1]:
from ipywidgets import interact, interactive, interact_manual, widgets, Label
from IPython.display import display, clear_output
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import gmaps
import sys

In [2]:
#import os
#print(os.getcwd()) # jupyter's folder

## Load secret.py
In **secret.py** I store my Google API key, and it's located in Data-Science's root folder.

In [3]:
sys.path.append('../../')
import secret
gmaps.configure(api_key=secret.GMAPS_KEY) # Your Google API key

## Define the dataset schema and load them

In [4]:
datasets_dir = '../datasets/'
limit = 200 # number of rows to read, -1 to read entire dataset

fields_to_drop = ['host_id', 'host_name', 'neighbourhood_group', 'neighbourhood', 'calculated_host_listings_count',
                  'availability_365', 'number_of_reviews', 'last_review', 'reviews_per_month']
listing = pd.read_csv(datasets_dir + 'listings.csv').head(limit).drop(fields_to_drop, axis=1)

fields_to_drop = ['summary', 'space', 'description', 'neighborhood_overview', 'transit', 'access', 'interaction',
                  'name', 'notes', 'house_rules', 'host_about', 'host_verifications', 'latitude', 'longitude',
                  'room_type', 'minimum_nights']

listing_detail = pd.read_csv(datasets_dir + 'listings.csv.gz',
                             dtype={'zipcode': str}).head(limit).drop(fields_to_drop, axis=1)

full_listing = pd.merge(listing, listing_detail, on='id')

## Set some parameters

In [5]:
# min and max price of initial rows
min_price = full_listing['price_x'].min(axis=0)
max_price = full_listing['price_x'].max(axis=0)

#print("min_price=%s, max_price=%s of %s rows" % (min_price, max_price, limit))

initial_rows = str(len(full_listing.index))

license_check = widgets.Checkbox(
    value=False,
    description='With license',
    disabled=False
)

## Define functions

In [6]:
def show_colorbar(label, inicio, num, final):
    fig = plt.figure(figsize=(3, 2))
    ax1 = fig.add_axes([0.5, 0.5, 0.9, 0.15])
    cmap = mpl.colors.ListedColormap(['green', 'red'])
    # If a ListedColormap is used, the length of the bounds array must be one greater
    # than the length of the color list.  The bounds must be monotonically increasing.
    bounds = [inicio, num, final]
    cb2 = mpl.colorbar.ColorbarBase(ax1, cmap=cmap, boundaries=bounds,
                                    spacing='proportional',  # TODO: fix this warning:
                                                             #RuntimeWarning: divide by zero encountered in double_scalars
                                    orientation='horizontal')
    cb2.set_label(label)
    plt.show()


def show_pie_chart(labels, sizes):
    # Pie chart, where the slices will be ordered and plotted counter-clockwise:
    fig1, ax1 = plt.subplots()
    ax1.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.show()


def show_analysis(max_price_filter):
    global m
    m.close()  # closing possible previous gmap
    m = gmaps.Map()

    info_box_template = """
<dl>
<dt>ID: {id}</dt><dd>{name}</dd>
<dt>Price</dt><dd>{price}</dd>
<dt>Property type and room type</dt><dd>{property_type}, {room_type}</dd>
<dt>Minimun nights</dt><dd>{minimum_nights}</dd>
<dt>Number of reviews</dt><dd>{number_of_reviews}</dd>
</dl>
"""
    apartment_info = []
    locations = []

    # Applying needed filters
    filtered_full_listing = full_listing.query('price_x <= %i' % max_price_filter)  # price

    if license_check.value == True:
        filtered_full_listing = filtered_full_listing.query('license == license')  # license


    # Once we have the filtered dataFrame, we get the needed info
    num_filtered_result = len(filtered_full_listing.index)
    print("INFO: Initial rows to process: {}, rows after filtering: {}".format(initial_rows, str(num_filtered_result)))
    
    if num_filtered_result == 0:
        sys.exit("No hay resultados para el filtro indicado")
    
    # license
    num_no_license = len(filtered_full_listing.query('license != license'))
    num_license = len(filtered_full_listing.query('license == license'))

    # Required minimun nights
    nights = filtered_full_listing['minimum_nights']
    nights_values = nights.value_counts()
    nights_index = [str(i) for i in nights_values.index]

    # Calculation of percentage (not needed because matplotlib will calculate it for us)
    #percentages = nights_values / num_filtered_result * 100

    # Showing all needed graphics
    # Matplotlib
    # Colorbar showing apartments wit/without license
    if num_license == 0:  # avoid exception
        num_license = num_filtered_result
    # TODO: fix showing red bar when having all licensed apartments
    show_colorbar('Licenses', 0, num_license, num_filtered_result)

    # Pie chart showing percentage of minimal nights to stay
    show_pie_chart(nights_index, nights_values)

    # GMAPS
    # Itering the filtered dataframe in order to feed gmaps info
    for row in filtered_full_listing.itertuples():
        apartment_info += [info_box_template.format(
            id=getattr(row, 'id'), name=getattr(row, 'name'), price=getattr(row, 'price_y'),
            property_type=getattr(row, 'property_type'), room_type=getattr(row, 'room_type'),
            minimum_nights=getattr(row, 'minimum_nights'), number_of_reviews=getattr(row, 'number_of_reviews'))]
        locations += [[getattr(row, 'latitude'), getattr(row, 'longitude')]]

    m = gmaps.Map()
    marker_layer = gmaps.marker_layer(locations, info_box_content=apartment_info)
    m.add_layer(marker_layer)
    display(m)

## Main program

In [7]:
# Show license's check
display(license_check)

# Create gmaps wigdet
m = gmaps.Map()

# Show interact button, in order to execute the price's filter and show gmaps widget
#interact(show_analysis, price = (min_price, max_price, 1))
interact_manual(show_analysis, max_price_filter = (min_price, max_price, 1))

<function __main__.show_analysis>